In [1]:
%matplotlib inline

# 中文分詞(Tensorflow+Keras)

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import pylab
import PIL  
import os
import pickle
import codecs
import glob
import math
import random
import builtins
import string
import numpy as np
import re
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.losses import *
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.applications import *
from keras.applications.imagenet_utils import *
from keras.optimizers import  *
from keras import regularizers


#如果你使用tensorflow+keras總是異常掛掉，請務必加入以下語法(貪心的tf把所有gpu全吃光一點不留給keras，變成全掛的慘劇)
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1
config.gpu_options.visible_device_list = "0"
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.90
set_session(tf.Session(config=config))

#run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

Using TensorFlow backend.


In [3]:
def str_full_to_half(in_str):
    out_str = []
    for char in in_str:
        inside_code = ord(char)
        if inside_code == 0x3000 or inside_code == 12288 or char==string.whitespace: # 全形空格直接轉換
             out_str.append(' ')
        elif inside_code >= 65281 and inside_code <= 65374:
            inside_code -= 0xfee0
            out_str.append(chr(inside_code))
        else:
            out_str.append(char)
        
    return ''.join(out_str)

In [4]:
as_train=codecs.open('../Data/ex12_train/as_training.utf8',encoding='utf-8-sig').read()
cityu_train=codecs.open('../Data/ex12_train/cityu_training.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_train=as_train.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_train=cityu_train.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data=as_train+'\r\n'+cityu_train #把兩個語料合併
data=data.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data=str_full_to_half(data) #把所有全形轉半形

raw_data_train=data.split('\r\n')#分行

raw_data_train=[row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_train] #移除分行字元

print(raw_data_train[:20])

train_data_len=[len(txt.split('|'))  for txt in  raw_data_train]
print(max(train_data_len))



['時間|:', '三月|十日|(|星期四|)|上午|十時|。', '地點|:', '學術|活動|中心|一樓|簡報室|。', '主講|:', '民族所|所長|莊英章|先生|。', '講題|:', '閩|、|台|漢人|社會|研究|的|若干|考察|。', '李|院長|於|二月|二十六日|至|三月|十五日|赴|美|訪問|,', '期間|將|與|在|美|院士|商討|院務|,', '與|美國|大學|聯繫|商討|長期|合作|事宜|,', '並|辦理|加州|大學|退休|等|手續|。', '出國|期間|院務|由|羅|副院長|代行|。', '總辦事處|秘書組|主任|戴政|先生|請辭|獲准|,', '所|遺|職務|自|三月|一日|起|由|近代史|研究所|研究員|陶英惠|先生|兼任|。', '植物|研究所|所長|周昌弘|先生|當選|第三世界|科學院|(|The|Third|World|Academy|of|Sciences|,', '簡稱|TWAS|)|院士|。', 'TWAS|係|一九八三年|由|Prof|Adbus|Salam|(|巴基斯坦籍|,', '曾|獲|諾貝爾獎|)|發起|成立|,', '會員|遍佈|63|個|國家|,']
211


In [5]:
as_test=codecs.open('../Data/ex12_train/as_testing_gold.utf8',encoding='utf-8-sig').read()
cityu_test=codecs.open('../Data/ex12_train/cityu_test_gold.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_test=as_test.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_test=cityu_test.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data_test=as_test+'\r\n'+cityu_test #把兩個語料合併
data_test=data_test.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data_test=str_full_to_half(data_test) #把所有全形轉半形

raw_data_test=data_test.split('\r\n')#分行

raw_data_test=[row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_test] #移除分行字元

print(raw_data_test[:20])
test_data_len=[len(txt.split('|'))  for txt in  raw_data_test]
print(max(test_data_len))

['許多|社區|長青|學苑|多|開設|有|書法|、|插花|、|土風舞班|,', '文山區|長青|學苑|則|有|個|十分|特別|的|「|英文|歌唱班|」|,', '成員|年齡|均|超過|六十|歲|,', '這|群|白髮蒼蒼|,', '爺爺|、|奶奶級|的|學員|唱起|英文|歌|來|字正腔圓|,', '有模有樣|。', '對|他們|來說|,', '能|與|一|群|志同道合|的|朋友|共同|回味|年少|時期|流行|的|歌曲|,', '才|是|參加|英文|歌唱班|最|大|樂趣|。', '長青|學苑|英文|歌唱班|昨日|在|社教館|舉行|「|英文|懷念|金曲|演唱會|」|,', '曲目|包括|「|大江東去|」|、|「|月河|」|、|「|真善美|」|等|大眾|耳熟能詳|的|英文|歌曲|。', '難得|公開|演唱|,', '這些|有|著|豐富|人生|閱歷|的|學員|絲毫|不|覺得|緊張|怯場|,', '只|見|台|上|唱|得|盡興|,', '台|下|不少|聽眾|也|一時|技癢|跟|著|唱和|起來|。', '長青|學苑|英文|歌唱班|成立|至今|已|兩|年|,', '目前|成員|約|廿五|人|,', '年齡|都|在|六十|歲|以上|,', '其中|以|軍公教|退休|人員|居多|,', '並|有|現任|大學|教授|,']
104


In [6]:
vocabs=sorted(set(list(''.join(raw_data_train))))
vocabs.remove('|')
vocabs.insert(0,'/Unknow')
vocabs.insert(0,'<PAD>')

print(len(vocabs))
print(vocabs[:100])

6296
['<PAD>', '/Unknow', ' ', '!', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '°', '·', '×', 'ˊ', 'ˋ', '˙', 'Α', 'Β', 'Γ', 'Ε', 'Η']


In [7]:
char_to_index=dict((w, i) for i, w in enumerate(vocabs))
index_to_char=dict((i, w) for i, w in enumerate(vocabs))

print({key:char_to_index[key] for key in list(char_to_index.keys())[:100]})

{'蜢': 4777, '碉': 3747, '諾': 5047, '逍': 5378, '衍': 4842, '湖': 2991, '敉': 2280, '鈞': 5566, '遞': 5425, '裟': 4883, '妨': 1267, '讞': 5095, '炘': 3155, '哩': 850, '借': 412, '宴': 1419, '意': 1871, '蹦': 5261, '旬': 2339, '磨': 3777, '凌': 550, '賠': 5161, '撞': 2210, '●': 137, '戛': 1968, '惴': 1861, '囝': 1046, '魄': 6088, '楣': 2614, '寓': 1433, '篋': 3976, '洫': 2891, '瀋': 3119, '嚨': 1027, '壯': 1196, '蟬': 4819, '蕈': 4656, '卒': 685, 'ㄟ': 182, '捭': 2097, '潔': 3067, '枺': 2501, '捻': 2104, '鉅': 5577, '搓': 2159, '暝': 2403, '霑': 5843, '尾': 1475, '棋': 2572, '幻': 1634, '迸': 5368, '圓': 1065, '噬': 1008, '凝': 553, '熗': 3219, '益': 3619, '胳': 4333, '鐺': 5696, '到': 584, '嚅': 1017, '催': 451, '下': 207, '蟳': 4821, 'z': 86, '好': 1250, '鷸': 6200, '荷': 4543, '件': 298, '鞠': 5882, '膾': 4394, '推': 2127, '鏈': 5673, '赧': 5186, '蠅': 4825, '萄': 4582, '筵': 3952, '呷': 802, '﹑': 6286, '鯤': 6109, '箐': 3958, '滅': 3021, '奘': 1235, '武': 2733, '瘴': 3570, '瀨': 3132, '絞': 4094, '封': 1452, '玕': 3357, '珍': 3376, '風': 5943, '鯰': 6112, '祛': 3806, '

In [8]:
idx_train=0
idxs_train=np.arange(len(raw_data_train))
np.random.shuffle(idxs_train)

idx_test=0
idxs_test=np.arange(len(raw_data_test))
np.random.shuffle(idxs_test)

def get_next_minibatch(minibatch_size,is_train=True):
    global idx_train,idxs_train,raw_data_train,idx_test,idxs_test,raw_data_test

    groundtruths=[]
    idx=idx_train
    idxs=idxs_train
    raw_data=raw_data_train
    if is_train==False:
        idx=idx_test
        idxs=idxs_test
        raw_data=raw_data_test
    np.random.shuffle(raw_data)
    #定義輸出向量形狀
    feature_arr=np.zeros((minibatch_size,128))
    label_arr=np.zeros((minibatch_size,128,4))
    length_arr=np.zeros((minibatch_size))
    batch=0
    while batch<minibatch_size:
        seq_len=len(raw_data[idxs[idx]].replace('|',''))
        if seq_len<=128 and seq_len>0:
            length_arr[batch]=seq_len
            
            groundtruth_seq=[]
            words=raw_data[idxs[idx]].split('|') #轉換成分詞後的詞清單

            pos=0
            #BMES=>[0,1,2,3]
            for word in words:
                for i in range(len(word)):
                    #如果在字典中則取出其索引
                    if word[i] in char_to_index:
                        feature_arr[batch,pos]=char_to_index[word[i]]
                    #否則定為未知
                    else:
                        feature_arr[batch,pos]=char_to_index['/Unknow']

                    #轉換為BMES
                    if len(word)==1 and i==0: #S 自己就是一個單詞
                        label_arr[batch,pos,3]=1
                    elif i==0: #B 是一個詞的開始
                        label_arr[batch,pos,0]=1
                    elif i==len(word)-1:  #E 是一個詞的結束
                        label_arr[batch,pos,2]=1
                    else: #M 是一個詞的中間
                        label_arr[batch,pos,1]=1  
                    pos+=1

                groundtruth_seq.append(word)
            groundtruths.append(groundtruth_seq)
            batch+=1
        idx+=1
        if idx>len(idxs):
            idx=0
            np.random.shuffle(idxs)
    idx_train=idx
    if is_train==False:
        idx_test=idx
    return feature_arr.astype(np.float32),label_arr.astype(np.float32),length_arr.astype(np.float32),groundtruths
        
x_feature,y_label,x_len,ground_truths=get_next_minibatch(2)
print(x_feature.shape)
print(y_label.shape)
print(x_len)
print(ground_truths)

(2, 128)
(2, 128, 4)
[13. 12.]
[['可是', '我們', '現在', '不', '知道', '為什麼', '﹖'], ['就', '能', '注意到', '這', '座', '山', '的', '特色', ',']]


In [ ]:

print('Build model...')


model = Sequential()
model.add(Embedding(len(vocabs),256,input_length=128,mask_zero=True))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Dense(4))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
if not os.path.exists('Models'):
    os.mkdir('Models')
    print("Directory Models Created ")
    
if os.path.exists('Models/word_segment_keras.h5'):
    model.load_weights('Models/word_segment_keras.h5')
    print('load model')
model.summary()




Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 256)          1611776   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 512)          1050624   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128, 512)          1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 512)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 128, 4)            2052      
_________________________________________________________________
activation_1 (Activation)    (None, 128, 4)            0         
Total params: 4,239,364
Trainable params: 4,239,364
Non-trainable params: 0
___________________________________________________

In [ ]:
num_epochs=3
minibatch_size=32
for epoch in range(num_epochs):
    mbs = 0
    rows = 0
    while mbs <1000:
        try:
            raw_features, raw_label,raw_len,raw_ground_truths = get_next_minibatch(minibatch_size)
            
            loss=model.train_on_batch(raw_features, raw_label)
            #每100次存一次檔
            if (mbs+1) % 50 == 0 :
                print('epoch {0}/{1} {2}-{3}  loss:{4:.3} accuracy:{5:.2%}  '.format(epoch,num_epochs,mbs+1-50,mbs+1,loss[0],loss[1]))
                model.save_weights('Models/word_segment_keras.hdf5')
            if mbs%200==0 :
                features_test, labels_test,len_test,ground_truths_test=get_next_minibatch(minibatch_size,False)
                print('-----測試集驗證--------')
                result=model.predict(features_test)
                for i in range(3):
                    answer='|'.join(ground_truths_test[i])+'|'
                    pred=[]
                    words=list(''.join(ground_truths_test[i]))
                    for j in range(len(words)):
                        word=words[j]
                        onehot=np.argmax(result[i][j])
                        if onehot>=2:
                            pred.append(word+'|')
                        else:
                            pred.append(word)

                    pred=''.join(pred)  
                    print('predict:'+pred)
                    print('answer :'+answer)
                print('-----------------')
        except Exception as e:
            print(e)
        mbs += 1

    
    model.save_weights('Models/word_segment_keras.hdf5')

-----測試集驗證--------
predict:長|實|(|0|0|0|1|)執|行|董|事|趙|國|雄|指|出|,|樓|市目前仍處鞏固期,發展商再度提供優|惠|或|二|按等促銷樓|盤|是|可以|理|解|的|。|
answer :長實|(|0001|)|執行|董事|趙國雄|指出|,|樓市|目前|仍|處|鞏固期|,|發展商|再度|提供|優惠|或|二按|等|促銷|樓盤|是|可以|理解|的|。|
predict:十|一|年|之|間|成|長|五|點|三|倍|;|
answer :十一|年|之間|成長|五點三|倍|;|
predict:「小森|先|生|,|
answer :「|小森|先生|,|
-----------------
